In [1]:
import pyspark

In [4]:
# sc = pyspark.SparkContext()
print sc

<SparkContext master=local[*] appName=PySparkShell>


In [5]:
print sc.appName
print sc.applicationId

PySparkShell
local-1517355918427


In [8]:
print sc.sparkHome
sc.sparkUser()

None


u'mprakash'

In [9]:
sc.uiWebUrl

u'http://10.0.4.101:4040'

In [14]:
from pyspark.sql import SparkSession, SQLContext


In [15]:
sqlcontext = SQLContext(sc)

In [16]:
spark=sqlContext.sparkSession

In [17]:
print spark

In [18]:
comm_data = spark.read.csv("../data/mp_comm_volume.csv", header="true",inferSchema="true")

In [19]:
comm_data.columns

['Comm_Date', 'ReceivedComms', 'SentComms']

In [21]:
comm_data.count()

444

In [22]:
comm_data.describe(['ReceivedComms']).show()

+-------+------------------+
|summary|     ReceivedComms|
+-------+------------------+
|  count|               444|
|   mean|19.186936936936938|
| stddev|17.226645081224586|
|    min|                 0|
|    max|                80|
+-------+------------------+



In [23]:
comm_data.describe(['SentComms']).show()

+-------+-----------------+
|summary|        SentComms|
+-------+-----------------+
|  count|              444|
|   mean|5.405405405405405|
| stddev|5.315556416393526|
|    min|                0|
|    max|               30|
+-------+-----------------+

